In [1]:
import pandas as pd
import numpy as np
import json
from Bio import SeqIO
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
#Load tree and sequence files
with open('augur_output/flu_seasonal_h3n2_ha_6y_tree.json', 'r') as jsonfile:
    tree_6y = json.load(jsonfile)

with open('augur_output/flu_seasonal_h3n2_ha_6y_sequences.json', 'r') as jsonfile:
    seq_6y = json.load(jsonfile)

#Load genbank file with reference HA sequence
ref_names = {'hemagglutinin':'hemagglutinin','Signal peptide':'SP_ref','HA1 protein':'HA1_ref', 'HA2 protein':'HA2_ref'}
ref_seqs = {}

for seq_record in SeqIO.parse('h3n2_outgroup.gb', 'genbank'):
    for feature in seq_record.features:
        if feature.type=='CDS':
            ref_seqs[str(ref_names[feature.qualifiers['product'][0]])]= (feature.location.extract(seq_record).seq.translate())

In [91]:
#Input desired 0-based HA1 amino acid residue positions, find reference amino acid

positions = [160,194,186,225,219,203,156,138]
position_refaa = []

for pos in positions:
    position_refaa.append((str(pos-1), ref_seqs['HA1_ref'][pos-1]))
    
    
tip_muts = {}

def traverse(branch, seq, pos_list):

    if 'children' not in branch.keys():

        tip_muts[branch['strain']]=[branch['aa_muts']['HA1'], branch['aa_muts']['HA2'], 
                                    branch['aa_muts']['SigPep'],branch['attr']['num_date'], 
                                    branch['attr']['clade_membership']] + [str(seq[str(branch['clade'])]['HA1'][pos]) if pos in seq[str(branch['clade'])]['HA1'] else str(ref_aa) 
                                    for pos, ref_aa in pos_list]

    else:
        for child in branch['children']:
            traverse(child, seq, pos_list)

    
traverse(tree_6y, seq_6y, position_refaa)

#Organize data in a DF
df = pd.DataFrame(tip_muts).T
df.reset_index(inplace=True)
df.columns = ['strain', 'tip_HA1_muts', 'tip_HA2_muts', 'tip_SigPep_muts', 'date', 'clade']+positions
df['passage'] = np.select((df.strain.str.contains('egg'), df.strain.str.contains('cell')), ('egg', 'cell'))
df['source'] = np.select((df.passage=='egg', df.passage=='cell', df.passage=='0'), 
                         (df.strain.str.replace('-egg',''), df.strain.str.replace('-cell',''), df.strain))

#!!!! Must first check all sites of insterest to make sure clades have predeominantly one genotype!!!!
#At each position, find predominant genotype of circulating virus by clade
#Use this to determine whether egg-passaged strains have mutated
#!!!! Must first check all sites of insterest to make sure clades have predeominantly one genotype!!!!

clade_gtype = {}
for p in positions:
    clade_gtype_pos = {}
    for c_name, clade in df[df['passage']=='0'].groupby('clade'):
        clade_gtype_pos[c_name] = str(clade[p].value_counts().nlargest(1))[0]
    clade_gtype[p] = clade_gtype_pos

for p in positions:
    df['circulating'+str(p)] = df['clade'].map(clade_gtype[p])

#make tidy version of df where each mutation gets a row
mut_df = pd.DataFrame(columns=['mutation']+ list(df.columns))

count=0
for i, r in df.iterrows():

    for ha1_mut in r['tip_HA1_muts']:
        mut_df.loc[count]= ['HA1'+str(ha1_mut)] + list(df.loc[i])
        count+=1
        
    for ha2_mut in r['tip_HA2_muts']:
        mut_df.loc[count]= ['HA2'+str(ha2_mut)] + list(df.loc[i])
        count+=1
        
    for sp_mut in r['tip_SigPep_muts']:
        mut_df.loc[count]= ['SP'+str(sp_mut)] + list(df.loc[i])
        count+=1